In [110]:
import glob
import pandas as pd
import os 

In [111]:
game_sim_files = glob.glob('../output_results/game_sim/game_level/*/*/conducted*')

In [112]:
all_dfs = []
for f in game_sim_files:
    f_path, fname = os.path.split(f)
    f_path, trial_name = os.path.split(f_path)
    game_level = os.path.basename(f_path)

    df = pd.read_json(f, lines=True)
    df['trial_name'] = trial_name
    df['game_level'] = game_level
    all_dfs.append(df)

In [114]:
full_df = pd.concat(all_dfs)

In [115]:
print(full_df['info_items'].iloc[0])

Here are the key pieces of information extracted from the interview transcript:

Information item #1: North Korea has started rebuilding the facilities it uses to launch satellites into orbit, specifically the Sohae launch site, also known as Tongchang-ri, on the west coast of North Korea.

Information item #2: The rebuilding of the launch site may have started as early as mid-February, before the summit between President Trump and Kim Jong Un, or as recently as over the weekend, after the summit finished.

Information item #3: The launch site was previously dismantled by North Korea as a gesture of goodwill in the denuclearization process, but the rebuilding is seen as a potential response to the failed summit talks.

Information item #4: The Sohae launch site is not considered a significant concession by North Korea, as they can launch missiles from mobile launchers, and the site is considered outdated and not worth much.

Information item #5: The nuclear situation is more complicate

In [134]:
running_convo_df = pd.DataFrame(full_df['running_conversations'].to_list()).loc[lambda df: df['info_items_1'].isnull()]
running_convo_df[['trial_name', 'game_level']] = full_df[['trial_name', 'game_level']].reset_index(drop=True)

In [144]:
info_num_columns = list(filter(lambda x: 'info_item_num' in x , running_convo_df.columns))
info_desc_columns = list(filter(lambda x: 'info_items_desc' in x , running_convo_df.columns))
persuasion_columns = list(filter(lambda x: 'persuasion' in x , running_convo_df.columns))

In [145]:
running_convo_df[info_num_columns]

,info_item_num_1,info_item_num_2,info_item_num_3,info_item_num_4,info_item_num_5,info_item_num_6,info_item_num_7
500,[],[],[],[],[],[],[]
501,[],[],[],[],[],[],[]
502,[],[],[],[],[],[],[]
503,[],[],[],[],[10],[],[]
504,[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...
5545,[],[],[],[],[],[],[]
5546,[],[],[],[],[],[],[]
5547,[],[],[],[],[],[],[]
5548,[],[],[],[],[],[],[]


In [150]:
running_convo_df[persuasion_columns]

,persuasion_0,persuasion_1,persuasion_2,persuasion_3,persuasion_4,persuasion_5,persuasion_6,persuasion_7
500,None,1,1,1,1,1,1,1
501,None,2,2,2,2,2,2,2
502,None,2,2,2,2,2,2,2
503,None,2,2,2,2,2,3,3
504,None,1,2,3,4,4,4,4
...,...,...,...,...,...,...,...,...
5545,None,3,3,3,3,3,3,3
5546,None,3,3,3,3,3,3,3
5547,None,3,3,3,3,3,3,3
5548,None,3,3,3,3,3,3,3


In [138]:
running_convo_df['total_info_items'] = running_convo_df[info_items_columns].apply(lambda s: s.str.len()).fillna(0).sum(axis=1)

In [139]:
(running_convo_df
 .groupby(['trial_name', 'game_level'])
 ['total_info_items']
 .mean()
 .unstack()
)

game_level,game_level_advanced,game_level_basic,game_level_intermediate
trial_name,,,
interviewer_gpt-4o-mini_vs_source_gpt-4o,1.390,1.042222,1.1000
interviewer_gpt-4o_vs_source_gpt-4o,1.380,1.046667,1.0925
interviewer_meta-llama_Meta-Llama-3.1-70B-Instruct_vs_source_gpt-4o,1.075,0.918000,0.8900
interviewer_meta-llama_meta-llama-3.1-8b-instruct_vs_source_gpt-4o,NaN,0.890000,0.8860


In [128]:
running_convo_df.columns

Index(['question_0', 'persuasion_0', 'info_items_0', 'answer_0', 'question_1',
       'persuasion_1', 'info_items_1', 'answer_1', 'question_2',
       'persuasion_2', 'info_items_2', 'answer_2', 'question_3',
       'persuasion_3', 'info_items_3', 'answer_3', 'question_4',
       'persuasion_4', 'info_items_4', 'answer_4', 'question_5',
       'persuasion_5', 'info_items_5', 'answer_5', 'question_6',
       'persuasion_6', 'info_items_6', 'answer_6', 'question_7',
       'persuasion_7', 'info_items_7', 'answer_7', 'question_8', 'answer_8',
       'info_items_desc_1', 'info_item_num_1', 'info_items_desc_2',
       'info_item_num_2', 'info_items_desc_3', 'info_item_num_3',
       'info_items_desc_4', 'info_item_num_4', 'info_items_desc_5',
       'info_item_num_5', 'info_items_desc_6', 'info_item_num_6',
       'info_items_desc_7', 'info_item_num_7', 'trial_name', 'game_level',
       'total_info_items'],
      dtype='object')

In [127]:
running_convo_df[info_items_columns]

,info_items_0,info_items_1,info_items_2,info_items_3,info_items_4,info_items_5,info_items_6,info_items_7,info_items_desc_1,info_items_desc_2,info_items_desc_3,info_items_desc_4,info_items_desc_5,info_items_desc_6,info_items_desc_7
0,None,[All relevant information has been given to th...,[All relevant information has been given to th...,[All relevant information has been given to th...,[All relevant information has been given to th...,[All relevant information has been given to th...,[All relevant information has been given to th...,[All relevant information has been given to th...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,[All relevant information has been given to th...,[All relevant information has been given to th...,[All relevant information has been given to th...,[All relevant information has been given to th...,[All relevant information has been given to th...,[All relevant information has been given to th...,[All relevant information has been given to th...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,[All relevant information has been given to th...,[All relevant information has been given to th...,[All relevant information has been given to th...,[All relevant information has been given to th...,[All relevant information has been given to th...,[All relevant information has been given to th...,[All relevant information has been given to th...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,None,[All relevant information has been given to th...,[All relevant information has been given to th...,[All relevant information has been given to th...,[All relevant information has been given to th...,[All relevant information has been given to th...,[All relevant information has been given to th...,[No relevant information for this question. Fe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,None,[All relevant information has been given to th...,[All relevant information has been given to th...,[All relevant information has been given to th...,[All relevant information has been given to th...,[All relevant information has been given to th...,[All relevant information has been given to th...,[All relevant information has been given to th...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5545,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All relevant information has been given to the...,All relevant information has been given to the...,All relevant information has been given to the...,All relevant information has been given to the...,All relevant information has been given to the...,All relevant information has been given to the...,All relevant information has been given to the...
5546,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All relevant information has been given to the...,All relevant information has been given to the...,All relevant information has been given to the...,All relevant information has been given to the...,All relevant information has been given to the...,All relevant information has been given to the...,All relevant information has been given to the...
5547,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All relevant information has been given to the...,All relevant information has been given to the...,All relevant information has been given to the...,All relevant information has been given to the...,All relevant information has been given to the...,All relevant information has been given to the...,All relevant information has been given to the...
5548,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All relevant information has been given to the...,All relevant information has been given to the...,All relevant information has been given to the...,All relevant information has been given to the...,All relevant information has been given to the...,All relevant information has been given to the...,All relevant information has been given to the...


# Look at sample DF

In [153]:
df = pd.read_json('../test/conducted_interviews_batch_0_2.jsonl', lines=True)

In [156]:
pd.DataFrame(df['running_conversations'].tolist())

,question_0,persuasion_0,info_items_0,answer_0,question_1,persuasion_1,info_items_1,answer_1,question_2,persuasion_2,info_items_2,answer_2,question_3,persuasion_3,info_items_3,answer_3,question_4,answer_4
0,"Today, we have the opportunity to speak with a...",None,None,Thank you for the introduction. It's always in...,“What has been the predominant reaction within...,1,[No relevant information for this question. Fe...,"Well, you know, reactions can vary widely. Som...","“I understand the complexity, but could you sh...",1,[Information Item #4: The Russian government i...,"Well, there hasn't been a major outcry from th...",“I appreciate your insights on the broader con...,2,[All relevant information has been given to th...,"You know, there may be some impact on public p...",Thank you for providing your valuable insights...,Thank you for having me. It’s always great to ...
1,"Today, we're diving into the final year of the...",None,None,Thank you for the introduction. I'm glad to be...,"“In your opinion, what do you think will be th...",1,[Information Item #4: President Bush will like...,"In my view, the most significant aspect of Pre...",“Looking at the upcoming foreign policy decisi...,1,[Information Item #3: President Bush is planni...,Certainly. I anticipate that the current admin...,"“In your opinion, what may be some potential s...",1,[No relevant information for this question. Fe...,While I don’t have specific information on sur...,Thank you for sharing your insights and analys...,Thank you for the opportunity to discuss these...
